# Overview #
In this notebook, you leverage the cpdctl command to copy assets from one deployment space of Cloud Pak for Data to another deployment space. This is typically applicable in MLOps process where you want to promote the relevant data assets and AI models from an initial deployment space, such as Pre-Prod (or UAT) deployment space, to another deployment space, such as Prod deployment space.

For purposes of AI governance as well as CI/CD component of MLOps, it is important to be able to automate the process of promoting relevant assets from one deployment space owner by development team to another deployment space owned by the QA team.

Please note that the two deployment spaces could exist on the same Cloud Pak for Data cluster or could belong to two different Cloud Pak for Data clusters in different environments.

### Execution Steps ###
In order to leverage cpdctl to copy assets from one deployment space to another, you need to provide the following information:
- SOURCE_CPD_URL : The url for the source Cloud Pak for Data cluster
- SOURCE_CPD_USERNAME: The username for the source Cloud Pak for Data cluster
- SOURCE_CPD_PASSWORD: The password for the source Cloud Pak for Data cluster
- TARGET_CPD_URL : The url for the target Cloud Pak for Data cluster
- TARGET_CPD_USERNAME: The username for the target Cloud Pak for Data cluster
- TARGET_CPD_PASSWORD: The password for the target Cloud Pak for Data cluster
- SOURCE_DEPLOYMENT_SPACE_NAME: The name of the deployment space on the source Cloud Pak for Data cluster (source deployment space should exist).
- TARGET_DEPLOYMENT_SPACE_NAME: The name of the deployment space on the target Cloud Pak for Data cluster (any name you choose since it will be created).
- TARGET_MODEL_NAME: The name of the model in the atarget deployment space (any name you choose since it will be created).

Given this information, this notebook will define the cpdctl contexts corresponding to the source and target Cloud Pak for Data clusters, create a new deployment space on the target cluster, and copy all assets from the source deployment space to the target deployment space.

The code assumes that the SOURCE_DEPLOYMENT_SPACE_NAME exists and contains the relevant assets and that the TARGET_DEPLOYMENT_SPACE_NAME also exists but has no assets.

In [ ]:
# Import required libraries and modules
import base64
import json
import os
import platform
import requests
import tarfile
import zipfile
import jmespath
import subprocess
from IPython.core.display import display, HTML

## Credentials for Cloud Pak for Data and Deployment Spaces 
Please update the information below to include the Deployment Spaces and credentials for Cloud Pak for Data. The provided credentials should have the permissions to edit and update deployment spaces. Typically, **datascientist** user with Data Scientist role can have the right permissions to manage deployment spaces.

For the Cloud Pak for Data url, if you are running on an environment hosted on Skytap, make sure to use the internal URL. You can get the correct url by running the following command in the openshift cluster.
**oc get ZenService lite-cr -o jsonpath="{.status.url}{'\n'}"**

Alternatively, you can log into the OpenShift web consolve, navigate to Networking ==> Routes and find the Cloud Pak for Data url under Location.

In [ ]:
SOURCE_CPD_USERNAME = '<USERNAME_WITH_ACCESS_TO_SOURCE_DEPLOYMENT_SPACE>' # for example: datascientist
SOURCE_CPD_PASSWORD = '<PASSWORD_FOR_SOURCE_CPD_USERNAME>'
SOURCE_CPD_URL = '<CLOUD PAK FOR DATA URL FOR SOURCE CLUSTER>' #typically, this would be https://cpd-cpd-instance.apps.demo.ibmdte.net

TARGET_CPD_USERNAME = '<USERNAME_WITH_ACCESS_TO_TARGET_DEPLOYMENT_SPACE>' # for example: datascientist
TARGET_CPD_PASSWORD = '<PASSWORD_FOR_TARGET_CPD_USERNAME>'
TARGET_CPD_URL = '<CLOUD PAK FOR DATA URL FOR TARGET CLUSTER>' #typically, this would be https://cpd-cpd-instance.apps.demo.ibmdte.net

# Code assumes both deployment spaces exist
SOURCE_DEPLOYMENT_SPACE_NAME='<NAME OF SOURCE DEPLOYMENT SPACE>' # For example: 'churnUATspace'
TARGET_DEPLOYMENT_SPACE_NAME='<NAME OF TARGET DEPLOYMENT SPACE>' # For example: 'churn_prod_space'

TARGET_MODEL_NAME='<TARGET DEPLOYMENT NAME>' # Any name to assign to target deployment, for example; ChurnPredictionProd

### Install the version v1.0.105 of `cpdctl`


In [ ]:
PLATFORM = platform.system().lower()
CPDCTL_ARCH = "{}_amd64".format(PLATFORM)
CPDCTL_RELEASES_URL="https://api.github.com/repos/IBM/cpdctl/releases"
CWD = os.getcwd()
PATH = os.environ['PATH']
CPDCONFIG = os.path.join(CWD, '.cpdctl.config.yml')
version='v1.0.105'

response = requests.get(CPDCTL_RELEASES_URL)
asset_version = next(a for a in response.json() if version==a['tag_name'])
#assets = response.json()[0]['assets']
assets=asset_version['assets']
platform_asset = next(a for a in assets if CPDCTL_ARCH in a['name'])
cpdctl_url = platform_asset['url']
cpdctl_file_name = platform_asset['name']

response = requests.get(cpdctl_url, headers={'Accept': 'application/octet-stream'})
with open(cpdctl_file_name, 'wb') as f:
    f.write(response.content)
    
display(HTML('<code>cpdctl</code> binary downloaded from: <a href="{}">{}</a>'.format(platform_asset['browser_download_url'], platform_asset['name'])))

In [ ]:
%%capture

%env PATH={CWD}:{PATH}
%env CPDCONFIG={CPDCONFIG}

In [ ]:
if cpdctl_file_name.endswith('tar.gz'):
    with tarfile.open(cpdctl_file_name, "r:gz") as tar:
        tar.extractall()
elif cpdctl_file_name.endswith('zip'):
    with zipfile.ZipFile(cpdctl_file_name, 'r') as zf:
        zf.extractall()

if CPDCONFIG and os.path.exists(CPDCONFIG):
    os.remove(CPDCONFIG)
    
version_r = ! cpdctl version
CPDCTL_VERSION = version_r.s

print("cpdctl version: {}".format(CPDCTL_VERSION))

In [ ]:
!which cpdctl

# CPDCTL Demo

AI Lifecycle automation using `cpdctl` CLI tool with one CPD 4.0 cluster and two different deployment spaces
- 'UAT': UAT or Pre-Prod Deployment Space
- 'PROD': Prod Deployment Space

### Add CPD 4.0 cluster configuration

Add "source_user" user to the `cpdctl` configuration

In [ ]:
! cpdctl config user set source_user_2 --username {SOURCE_CPD_USERNAME} --password {SOURCE_CPD_PASSWORD}

In [ ]:
! cpdctl config user list

Add "source" cluster to the `cpdctl` configuration

In [ ]:
! cpdctl config profile set source_2 --url {SOURCE_CPD_URL} --user source_user_2

Add "source" context to the `cpdctl` configuration

In [ ]:
! cpdctl config context set source --profile source_2 --user source_user_2

List available contexts

In [ ]:
! cpdctl config context list

In [ ]:
! cpdctl config context use source

In [ ]:
! cpdctl space list

List available projects in "source" context

In [ ]:
! cpdctl project list

### TARGET cluster configuration ###
In this case, it is the same CPD 4.0 cluster but we're keeping it in general below so you can point to other clusters

Add "target_user" user to the `cpdctl` configuration

In [ ]:
! cpdctl config user set target_user --username {TARGET_CPD_USERNAME} --password {TARGET_CPD_PASSWORD}

Add "target" cluster to the `cpdctl` configuration

In [ ]:
! cpdctl config profile set target --url {TARGET_CPD_URL} --user target_user

Add "target" context to the `cpdctl` configuration

In [ ]:
! cpdctl config context set target --profile target --user target_user

List available contexts

In [ ]:
! cpdctl config context list

In [ ]:
! cpdctl config context use target

In [ ]:
! cpdctl project list

## Get Deployment Space IDs

In [ ]:
def getSpaceID(name, cluster):
    cmd="cpdctl space list --context " + cluster + " --output json" + " --jmes-query \"resources[?entity.name == " + "'" + name + "'" + "].metadata.id\""  
    print("executing command: ", cmd)
    
    result = subprocess.getoutput(cmd)
    space_id=json.loads(result)
    if len(space_id) != 1:
        print("Error, found ", len(space_id), " spaces with the name: ", name)
    return space_id[0]


In [ ]:
source_deployment_spaceID=getSpaceID(SOURCE_DEPLOYMENT_SPACE_NAME, 'source')
print("Source Deployment Space ID: ", source_deployment_spaceID)

Export All Assets from the dev deploymentspace

In [ ]:
EXPORT = {
    'all_assets': True
}
EXPORT_JSON = json.dumps(EXPORT)
! cpdctl config context use source
result = ! cpdctl asset export start --space-id {source_deployment_spaceID} --assets '{EXPORT_JSON}' --name source-space-assets --output json --jmes-query "metadata.id"
EXPORT_ID = result.s
print("The new export with ID: {}".format(EXPORT_ID))

In [ ]:
! cpdctl asset export download --space-id {source_deployment_spaceID} --export-id {EXPORT_ID} --output-file source-space-assets.zip

In [ ]:
# Create New Space using the TARGET_DEPLOYMENT_SPACE_NAME in the TARGET_CPD_CLUSTER
! cpdctl config context use 'target'
#! cpdctl space create --name {TARGET_DEPLOYMENT_SPACE_NAME}

In [ ]:
TARGET_SPACE_ID=getSpaceID(TARGET_DEPLOYMENT_SPACE_NAME, 'target')
print("Target Deployment Space ID: ", TARGET_SPACE_ID)

In [ ]:
# Get list of spaces to confirm
! cpdctl space list --context target

In [ ]:
result = ! cpdctl asset import start --space-id {TARGET_SPACE_ID} --import-file source-space-assets.zip --output json --jmes-query "metadata.id" --raw-output
IMPORT_ID = result.s
print("The new import ID is: {}".format(IMPORT_ID))

In [ ]:
! cpdctl asset import get --space-id {TARGET_SPACE_ID} --import-id {IMPORT_ID}

In [ ]:
! cpdctl ml model list --space-id {TARGET_SPACE_ID}

In [ ]:
result = ! cpdctl ml model list --space-id {TARGET_SPACE_ID} --output json --jmes-query "resources[0].metadata.id" --raw-output
TARGET_MODEL_ID = result.s
print("TARGET model ID is: {}".format(TARGET_MODEL_ID))

In [ ]:
ASSET_JSON = json.dumps({"id": TARGET_MODEL_ID})
ONLINE_JSON = json.dumps({})

! cpdctl ml deployment create --space-id {TARGET_SPACE_ID} --asset '{ASSET_JSON}' --online '{ONLINE_JSON}' --name {TARGET_MODEL_NAME}

## Verify Deployment ##
At this point, a new model deployment should appear in your target deployment space.

1. Navigate to your target deployment space: Select the Navigation Menu (top left hamburger icon), right click on *Deployments*, and select *Open Link in New Tab*.
2. In the new tab, select the *Spaces* tab and click on the name of the target deployment space (for example, *qa*)
3. On the Deployments/\<target space name\> page, click on *Deployments* tab.
4. Verify the *TARGET_MODEL_NAME* appears in the list of deployed models. Click the *TARGET_MODEL_NAME* model.
5. On the Deployed model page, click the Test tab and provide a sample test to validate the model returns predictions as expected.

    

In [ ]:
cpdtoken=os.environ['USER_ACCESS_TOKEN']
wml_credentials = {
"token": cpdtoken,
"instance_id" : "openshift",
"url": os.environ['RUNTIME_ENV_APSX_URL'],
"version": "4.0"
}

from ibm_watson_machine_learning import APIClient
client = APIClient(wml_credentials)


In [ ]:
def getSpaceIDwml(wml_client,space_name):
    spaces = wml_client.spaces.get_details()['resources'];
    spaceList = next(item for item in spaces if item['entity']['name']==space_name)
    spaceID = spaceList['metadata']['id']
    return spaceID

In [ ]:
space_name=TARGET_DEPLOYMENT_SPACE_NAME
space_id = getSpaceIDwml(client,space_name)
print(space_id)
client.set.default_space(space_id)

In [ ]:
space_details=client.spaces.get_details(space_id)

In [ ]:
client.repository.list_models()

In [ ]:
def getModelDetails(wml_client,deployment_name):
    models = wml_client.deployments.get_details()['resources'];
    modelList = next(item for item in models if item['entity']['name']==deployment_name)
    #modelID = modelList['metadata']['id']
    #return modelID
    return modelList
    

In [ ]:
model_name=TARGET_MODEL_NAME
model_details = getModelDetails(client,model_name)
print(model_details)

In [ ]:
# Score the model on a test dataset
scoring_payload = {
    "input_data": [{
        'fields': ['ID', 'LONGDISTANCE', 'INTERNATIONAL', 'LOCAL', 'DROPPED', 'PAYMETHOD', 'LOCALBILLTYPE', 'LONGDISTANCEBILLTYPE', 'USAGE', 'RATEPLAN', 'GENDER','STATUS', 'CHILDREN', 'ESTINCOME', 'CAROWNER', 'AGE'],
        'values': [[1,28,0,60,0,"Auto","FreeLocal","Standard",89,4,"F","M",1,23000,"N",45]]}]
}



In [ ]:
modelID=model_details['metadata']['id']
predictions = client.deployments.score(modelID, scoring_payload)
print(json.dumps(predictions, indent=2))

# Summary #
This notebook illustrates one approach to apply CI/CD against your models where you can automate continuous integration and delivery of models from UAT (or preProd) deployment space to production deployment space.